Problem representation

In [62]:
# def get_balanced_tp(supply, demand, costs, penalties = None):
#     total_supply = sum(supply)
#     total_demand = sum(demand)
    
#     if total_supply < total_demand:
#         if penalties is None:
#             raise Exception('Supply less than demand, penalties required')
#         new_supply = supply + [total_demand - total_supply]
#         new_costs = costs + [penalties]
#         return new_supply, demand, new_costs
#     if total_supply > total_demand:
#         new_demand = demand + [total_supply - total_demand]
#         new_costs = costs + [[0 for _ in demand]]
#         return supply, new_demand, new_costs
#     return supply, demand, costs

# supply = [40, 30]
# demand = [30, 50]
# costs = [
#     [3, 4],
#     [2, 4]
# ]
# penalties = [3, 1]
# get_balanced_tp(supply, demand, costs, penalties)
# ([40, 30, 10], [30, 50], [[3, 4], [2, 4], [3, 1]])

# supply = [40, 30]
# demand = [30, 30]
# costs = [
#     [3, 4],
#     [2, 4]
# ]
# get_balanced_tp(supply, demand, costs, penalties)
# ([40, 30], [30, 30, 10], [[3, 4], [2, 4], [0, 0]])

Initialize matrix

In [63]:
import numpy as np


# M = float('inf')
M = 999999

costs = np.array([[7, 5, 5, 0],
                 [3, 10, 10, M],
                 [3, 10, 10, 0],
                 [M, M, 0, 0]])
supply = np.array([30, 20, 80, 80])
demand = np.array([40, 40, 20, 110])

## Use NorthWest method to find our path

In [64]:
def north_west_corner(supply, demand):
    """
    North West Corner Method for finding path (initial solutions) of transportation problem
    """
    m = len(supply) 
    n = len(demand)    #m - number of suppliers, n- number of consumers
    supply_copy = supply.copy()
    demand_copy = demand.copy()
    i = 0
    j = 0
    path = []
    while len(path) < m + n - 1:  #m+n basic variables
        s = supply_copy[i]   
        d = demand_copy[j]
        v = min(s, d)  #get maximum possible value (minimum of s and d)
        supply_copy[i] -= v   #delete value from supply
        demand_copy[j] -= v   #delete value from demand
        path.append(((i, j), v))   #add to our path
        if supply_copy[i] == 0 and i < m - 1:  #we cant use this supplier anymore
            i += 1
        elif demand_copy[j] == 0 and j < n - 1:   #we cant use this consumer anymore
            j += 1
    return path


supply = np.array([30, 20, 80, 80])
demand = np.array([40, 40, 20, 110])
path = north_west_corner(supply, demand)
print(path)

[((0, 0), 30), ((1, 0), 10), ((1, 1), 10), ((2, 1), 30), ((2, 2), 20), ((2, 3), 30), ((3, 3), 80)]


## Get U collumn and V row 

In [65]:
def get_us_and_vs(path, costs):
    us = [None] * len(costs)
    vs = [None] * len(costs[0])
    
    #set ui=0 of the row that is assigned the most
    row_common = np.array([value[0][0] for value in path])
    unique, counts = np.unique(row_common, return_counts=True)
    most_common_row = unique[np.argmax(counts)]
    us[most_common_row] = 0
    
    path_copy = path.copy()
    while len(path_copy) > 0:     #while we dont calculate for all basic variables
        for index, bv in enumerate(path_copy):
            i, j = bv[0]         #coordinates of basic variable
            if us[i] is None and vs[j] is None: continue      #skip cells that dont have ui or vi assigned yet
                
            cost = costs[i][j]
            if us[i] is None:          #assign ui based on vi
                us[i] = cost - vs[j]
            else: 
                vs[j] = cost - us[i]     #assign vi based on ui
            path_copy.pop(index)        #dont take the basic variable into account anymore
            break
            
    return us, vs      


path = north_west_corner(supply, demand)
us, vs = get_us_and_vs(path, costs)
print("U collumn: \n", us, "\nV row:\n", vs)   

U collumn: 
 [4, 0, 0, 0] 
V row:
 [3, 10, 10, 0]


## Calculate the not basic variable values

In [66]:
def get_Cj(path, costs, us, vs):
    Cj = []
    for i, row in enumerate(costs):
        for j, cost in enumerate(row):
            non_basic = all([p[0] != i or p[1] != j for p, v in path])
            if non_basic:
                Cj.append(((i, j), cost - us[i] - vs[j] ))
    
    return Cj

path = north_west_corner(supply, demand)
us, vs = get_us_and_vs(path, costs)

get_Cj(path, costs, us, vs)

[((0, 1), -9),
 ((0, 2), -9),
 ((0, 3), -4),
 ((1, 2), 0),
 ((1, 3), 999999),
 ((2, 0), 0),
 ((3, 0), 999996),
 ((3, 1), 999989),
 ((3, 2), -10)]

## Check if we need another iteration (if we have coefficient that is negative )

In [67]:
def improvement(Cj):
    for p, v in Cj:
        if v < 0:
            return True
    return False
print(improvement(get_Cj(path, costs, us, vs)))

True


## If true, check position of entering variable (smallest value)

In [68]:
def get_entering_variable(Cj):
    return min(Cj, key=lambda x: x[1])[0]

entering_var = get_entering_variable(get_Cj(path, costs, us, vs))
print("Position of entering variable: ", entering_var)


Position of entering variable:  (3, 2)


## Chain rule of finding leaving variable and changing values

In [69]:
def get_possible_next_nodes(loop, not_visited):
    last_node = loop[-1]
    row_candidates = [n for n in not_visited if n[0] == last_node[0]]
    column_candidates = [n for n in not_visited if n[1] == last_node[1]]
    if len(loop) < 2:
        return row_candidates + column_candidates
    else:
        prev_node = loop[-2]
        row_move = prev_node[0] == last_node[0]
    if row_move: return column_candidates
    return row_candidates

In [70]:
def get_loop(bv_positions, ev_position):
    def inner(loop):
        if len(loop) > 3:
            can_be_closed = len(get_possible_next_nodes(loop, [ev_position])) == 1
            if can_be_closed: return loop
        
        not_visited = list(set(bv_positions) - set(loop))
        possible_next_nodes = get_possible_next_nodes(loop, not_visited)
        for next_node in possible_next_nodes:
            new_loop = inner(loop + [next_node])
            if new_loop: return new_loop
    
    return inner([ev_position])


path = north_west_corner(supply, demand)
us, vs = get_us_and_vs(path, costs)
Cj = get_Cj(path, costs, us, vs)
improvement(Cj)

entering_var = get_entering_variable(Cj)
loop = get_loop([p for p, v in path], entering_var)
print("loop of donors and recipients ", loop)

loop of donors and recipients  [(3, 2), (3, 3), (2, 3), (2, 2)]


In [71]:
def ChainRule(path, loop):
    path_copy = path.copy()
    donors = loop[1::2]
    recipients = loop[::2]
    
    #Check minimum value in the donors in the loop
    donors_path = [(coords, value) for coords, value in path_copy if coords in donors]
    recipients_path = [(coords, value) for coords, value in path_copy if coords in recipients]
    leaving_var, min_value = min(donors_path, key=lambda x: x[1])
        
    #Add the entering variable to the path
    path_copy.append((loop[0], 0))
    #Update the path values
    for i, ((x,y),z) in enumerate(path_copy):
        if (x,y) in recipients:
            path_copy[i] = ((x,y), z + min_value)
        if (x,y) in donors:
            path_copy[i] = ((x,y), z - min_value)
    #Remove the leaving variable 
    path_copy.remove((leaving_var, 0))
    
    return path_copy

print("New path:\n", ChainRule(path, loop))

New path:
 [((0, 0), 30), ((1, 0), 10), ((1, 1), 10), ((2, 1), 30), ((2, 3), 50), ((3, 3), 60), ((3, 2), 20)]


In [ ]:
def transportation_method(costs, supply, demand):
    num_iter = 0
    
    def iteration(path, num_iter):
        num_iter += 1
        us, vs = get_us_and_vs(path, costs)
        Cj = get_Cj(path, costs, us, vs)
        if improvement(Cj):
            entering = get_entering_variable(Cj)
            loop = get_loop([p for p, v in path], entering)
            return iteration(ChainRule(path, loop), num_iter)
        return path, num_iter
    
    path = north_west_corner(supply, demand)
    
    final_basic_variables, num_iter = iteration(path, num_iter)
    
    matrix = np.zeros((len(costs), len(costs[0])))
    Z = 0
    for (i, j), v in final_basic_variables:
        matrix[i][j] = v
        Z += (v * costs[i][j])

    return matrix, Z, num_iter

matrix, Z, num_iter = transportation_method(costs, supply, demand)
print("Final matrix:\n", matrix)
print("Final objective function Z: ", Z)
print(f"Done in {num_iter} iterations")

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (7, 2) + inhomogeneous part.

### PuLP verification

In [ ]:
import pulp as pl

def verify_with_pulp(cost, supply, demand):
    prob = pl.LpProblem("Transportation_Problem", pl.LpMinimize)
    rows, cols = cost.shape
    x = pl.LpVariable.dicts("x", (range(rows), range(cols)), 0, None, pl.LpInteger)

    prob += pl.lpSum(cost[i][j] * x[i][j] for i in range(rows) for j in range(cols))

    for i in range(rows):
        prob += pl.lpSum(x[i][j] for j in range(cols)) == supply[i]
    for j in range(cols):
        prob += pl.lpSum(x[i][j] for i in range(rows)) == demand[j]

    prob.solve()

    solution = np.zeros_like(cost, dtype=int)
    for i in range(rows):
        for j in range(cols):
            solution[i, j] = int(x[i][j].varValue)

    return solution, pl.value(prob.objective)

# Example verification:
pulp_solution, pulp_cost = verify_with_pulp(costs, supply, demand)
print("PuLP solution:\n", pulp_solution)
print("PuLP objective value:", pulp_cost)



PuLP solution:
 [[ 0 30  0  0]
 [10 10  0  0]
 [30  0  0 50]
 [ 0  0 20 60]]
PuLP objective value: 370.0
